In [ ]:
import json
import uuid
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd

- Import the CSV, then sanitize the column names.
- Adjust `csv_inpath` as appropriate.
- Rename columns for clarity.

In [ ]:
# Load the CSV-formatted use cases from a file.
csv_inpath = Path("data/input/autodesk-jira-tickets.csv")
df = pd.read_csv(csv_inpath)
renames = {
    "Generative AI Tool ": "generative_ai_tool",  # This cust. data has an extra space.
    "ticket number": "ticket_number",
    "jira ticket": "jira_ticket",
    "Status": "status",
    "Prompts": "prompts",
    "Purpose of Use": "purpose_of_use",
    "Response to Request": "response_to_request",
}
df.rename(columns=renames, inplace=True)
# df.head()  # Uncomment to check input. Be sure not to share sensitive data.

Iterate down the rows of the CSV, transforming each one into a JSON object defined by [`use-case-schema.json`](./use-case-schema.json).

In [ ]:
status_mapping = {"Approved": 1, "Pending": 0, "Rejected": -1}
use_cases = []
now_iso = datetime.now(timezone.utc).isoformat()

# Row by row, transform each CSV-formatted use case into a JSON-formatted dict.
for _, row in df.iterrows():
    use_case = {
        "id": str(uuid.uuid4()),
        "description": (
            f"Purpose of Use:\n{row['purpose_of_use']}\n\n"
            f"Response to Request:\n{row['response_to_request']}\n\n"
            f"Prompts:\n{row['Prompts ']}\n\n"
            f"Ticket Number: {row['ticket_number']}\n"
            f"JIRA Ticket: {row['jira_ticket']}"
        ),
        "name": row["generative_ai_tool"].strip(),
        "icon": None,
        "ai_type": "",
        "governance_status": status_mapping.get(row["status"].strip(), 0),
        "domains": [],
        "industries": [],
        "regions": [],
        "risk_category_level": None,
        "custom_fields": [],
        "questionnaires": [],
        "inserted_at": now_iso,
        "updated_at": now_iso,
    }
    use_cases.append(use_case)

# Save the list of JSON-formatted (dict) use cases to a file.
outfolder = csv_inpath.parents[1] / "output"
outfolder.mkdir(parents=True, exist_ok=True)
filename = csv_inpath.with_name(csv_inpath.stem + "-bulk-upload.json").name
json_outpath = outfolder / filename
with open(json_outpath, "w") as f:
    json.dump(use_cases, f, indent=4)